In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, json

In [2]:
# Check GPU availability
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

True
True


In [3]:
DATAPATH = 'data'

In [4]:
# load metadata
csv_filepath ="/".join([DATAPATH, 'metadata.csv'])
df = pd.read_csv(csv_filepath)

In [5]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [6]:
# Check shape
df.shape

(45774, 17)

In [7]:
df[df['sha'].notna() == True].shape

(31753, 17)

In [8]:
df_ft = df[df['has_full_text'] == True]
df_ft.shape

(31753, 17)

In [9]:
df_ft[df_ft['sha'].notna()==False].shape

(0, 17)

In [10]:
df_ft['sha'][df_ft['sha'].duplicated()]

27528    45e40b2d7d973ed5c9798da613fb3cfa4427e2e2
31108    f3aafdecdc43a3f57e58cf6dcea038b1834a953e
33297    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33298    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33325    ba4afe00e152de82121a4445aed52c46833d927e
41028    4644c32551fb23aa873a7738ecc8d777bd49877e
41029    4644c32551fb23aa873a7738ecc8d777bd49877e
41030    4644c32551fb23aa873a7738ecc8d777bd49877e
44847    58be092086c74c58e9067121a6ba4836468e7ec3
Name: sha, dtype: object